In [1]:
import pandas as pd
import numpy as np
import csv
from decimal import Decimal


In [2]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("trg.csv")
df = df.rename({'class':'target'},axis=1)
training, testing = train_test_split(df, test_size=0.3, random_state=42)



In [3]:
vocab = {word for para in training['abstract'] for word in para.split()}


prior_probability = training['target'].value_counts(normalize=True).to_dict()
for key, value in prior_probability.items():
    prior_probability[key] = Decimal(str(value))
prior_probability 

{'E': Decimal('0.5457142857142857'),
 'B': Decimal('0.38785714285714284'),
 'A': Decimal('0.03535714285714286'),
 'V': Decimal('0.031071428571428573')}

In [4]:
word_orrurance = dict()
for row in training.itertuples(index = 0): 
    line = row.abstract
    target = row.target
    # print(target)
    for word in line.split():
        
        if word in word_orrurance.keys():
            if target in word_orrurance[word].keys():
                word_orrurance[word][target] += 1
            else:
                word_orrurance[word][target] = 1
        else:
            word_orrurance[word] = {"B": Decimal(0), "E":Decimal(0), "A":Decimal(0), "V":Decimal(0)}
            # word_orrurance[word] = {target:1}
            # word_orrurance[word] = {"B": 0, "E":0 , "A":0, "V":0}
            word_orrurance[word][target] += 1



data = []
for word, targets in word_orrurance.items():
    
    row = {'word': word}
    
    for i, (target, count) in enumerate(targets.items()):
        row [target] = count
    # append the row to the data list
    data.append(row)
word_orrurance_table = pd.DataFrame(data)
word_orrurance_table.fillna(0, inplace=True)



word_orrurance


{'the': {'B': Decimal('12290'),
  'E': Decimal('17525'),
  'A': Decimal('1342'),
  'V': Decimal('1214')},
 'structure': {'B': Decimal('120'),
  'E': Decimal('289'),
  'A': Decimal('28'),
  'V': Decimal('19')},
 'and': {'B': Decimal('6744'),
  'E': Decimal('8996'),
  'A': Decimal('766'),
  'V': Decimal('512')},
 'expression': {'B': Decimal('176'),
  'E': Decimal('743'),
  'A': Decimal('7'),
  'V': Decimal('9')},
 'of': {'B': Decimal('10883'),
  'E': Decimal('12756'),
  'A': Decimal('987'),
  'V': Decimal('886')},
 'alpha-amylase-encoding': {'B': Decimal('0'),
  'E': Decimal('3'),
  'A': Decimal('0'),
  'V': Decimal('0')},
 'gene': {'B': Decimal('1317'),
  'E': Decimal('1921'),
  'A': Decimal('173'),
  'V': Decimal('98')},
 'ramy2a': {'B': Decimal('0'),
  'E': Decimal('2'),
  'A': Decimal('0'),
  'V': Decimal('0')},
 'are': {'B': Decimal('1385'),
  'E': Decimal('1709'),
  'A': Decimal('326'),
  'V': Decimal('95')},
 'described': {'B': Decimal('39'),
  'E': Decimal('128'),
  'A': Decimal(

In [5]:
total_B = word_orrurance_table['B'].sum()
total_E = word_orrurance_table['E'].sum()
total_A = word_orrurance_table['A'].sum()
total_V = word_orrurance_table['V'].sum()

# conditional_probability = word_orrurance_table.copy()
# conditional_probability['B'] = (conditional_probability['B'] / (total_B)).apply(Decimal)
# conditional_probability['E'] = (conditional_probability['E'] / (total_E)).apply(Decimal)
# conditional_probability['A'] = (conditional_probability['A'] / (total_A)).apply(Decimal)
# conditional_probability['V'] = (conditional_probability['V'] / (total_V)).apply(Decimal)


conditional_probability_dict = word_orrurance

# for keys in conditional_probability_dict

In [6]:
def naive_Bayes(vocab, prior_probability, conditional_probability_dict,testing):
    predictions = []
    row_nm = 0
    for row in testing.itertuples(index = 0):
        row_nm += 1
        # print(row_nm)
        line = row.abstract
        prediction = predict(vocab,prior_probability,conditional_probability_dict,line)
        # print(prediction, row.target)
        predictions.append(prediction)
    testing['prediction'] = predictions
    return predictions

def predict(vocab,prior_probability,conditional_probability_dict,line):
    words = line.split()
    # print(words)
    prob_B = prior_probability['B'].ln()
    prob_E = prior_probability['E'].ln()
    prob_A = prior_probability['A'].ln()
    prob_V = prior_probability['V'].ln()
    for word in words:
        if word in vocab:
            prob_B += (conditional_probability_dict[word]['B'] + 1).ln() - (Decimal(total_B + len(vocab))).ln()
            prob_A += (conditional_probability_dict[word]['A'] + 1).ln() - (Decimal(total_A + len(vocab))).ln()
            prob_V += (conditional_probability_dict[word]['V'] + 1).ln() - (Decimal(total_V + len(vocab))).ln()
            prob_E += (conditional_probability_dict[word]['E'] + 1).ln() - (Decimal(total_E + len(vocab))).ln()
    # print(prob_B,prob_E,prob_A,prob_V)
    if prob_B > prob_E and prob_B > prob_A and prob_B > prob_V:
        return 'B'
    elif prob_E > prob_B and prob_E > prob_A and prob_E > prob_V:
        return 'E'
    elif prob_A > prob_B and prob_A > prob_E and prob_A > prob_V:
        return 'A'
    elif prob_V > prob_B and prob_V > prob_E and prob_V > prob_A:
        return 'V'
    else:
        return 'E'
# naive_Bayes(vocab,conditional_probability, prior_probability,conditional_probability_dict, testing)
# training_acc = (testing['target'] == testing['prediction']).mean()
# print(f"Training acc: {training_acc:.3f}")


In [7]:
test_df = pd.read_csv("tst.csv")
# test_df['class'] = test_df['abstract'].apply(naive_Bayes, args=(vocab, prior_probability, word_count))
# tst_prediction = test_df[['id', 'class']]
# tst_prediction.to_csv('tst_prediction.csv', index=False)
prediction_row = naive_Bayes(vocab, prior_probability,conditional_probability_dict, test_df)
print(prediction_row)
with open("tst.csv","rt") as source:
    rdr = csv.reader(source)
    c = -1
    with open("raar519.csv","wt", newline='') as result:
        wtr = csv.writer( result )
        for r in rdr:
            if (c==-1): wtr.writerow( (r[0], "class") ) #add csv header
            else: wtr.writerow( (r[0], prediction_row[c]) )
            c+=1

['B', 'E', 'E', 'E', 'E', 'E', 'E', 'B', 'E', 'B', 'A', 'E', 'B', 'B', 'E', 'E', 'E', 'E', 'E', 'B', 'B', 'E', 'B', 'B', 'E', 'B', 'E', 'B', 'B', 'E', 'E', 'B', 'B', 'E', 'B', 'V', 'E', 'E', 'B', 'E', 'E', 'E', 'B', 'E', 'E', 'E', 'E', 'B', 'B', 'E', 'E', 'E', 'E', 'B', 'B', 'E', 'E', 'B', 'B', 'B', 'E', 'E', 'E', 'B', 'E', 'B', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'E', 'B', 'E', 'E', 'E', 'E', 'E', 'E', 'B', 'E', 'E', 'E', 'B', 'B', 'E', 'E', 'B', 'B', 'B', 'B', 'B', 'B', 'E', 'E', 'E', 'B', 'B', 'E', 'B', 'B', 'B', 'E', 'E', 'B', 'B', 'B', 'B', 'B', 'E', 'E', 'B', 'E', 'E', 'E', 'B', 'B', 'B', 'E', 'B', 'B', 'E', 'E', 'B', 'E', 'E', 'B', 'B', 'E', 'B', 'B', 'E', 'B', 'E', 'E', 'B', 'E', 'E', 'E', 'B', 'E', 'E', 'B', 'E', 'E', 'E', 'B', 'B', 'A', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'E', 'B', 'E', 'E', 'E', 'E', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'B', 'E', 'E', 'B', 'B', 'E', 'A', 'B', 'B', 'E', 'B', 'A', 'B', 'B', 'B', 'E',